<a href="https://colab.research.google.com/github/perlitz/VAE_talk/blob/master/VAE_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install ipdb

import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import ipdb
import PIL
import torch.nn.functional as F

plt.ion()   # interactive mode

set_seed = 1
torch.manual_seed(set_seed);

In [0]:
batch_size_train = 64
batch_size_test  = 1000

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=False)

In [0]:
class Net(nn.Module):
  
  def __init__(self):
    """This is where I will define the layers I use"""
    super(Net, self).__init__()
    
    self.conv1 = nn.Conv2d(in_channels=1 , out_channels=10, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()

    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50 , 10)


  def forward(self, x):
    """This is where the forward pass is defined"""
    # ipdb.set_trace()

    x = self.conv1(x)
    x = F.max_pool2d(x, 2)
    x = F.relu6(x)

    x = self.conv2(x)
    x = self.conv2_drop(x)
    x = F.max_pool2d(x, 2)
    x = F.relu6(x)

    x = x.view(-1, 320)

    x = self.fc1(x)
    x = F.dropout(x, training=self.training)
    
    x = self.fc2(x)
    
    return F.log_softmax(x)

In [0]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr=0.01,momentum=0.5)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):

  net.train() # put the network in to training mode  


  for batch_idx , (data, label) in enumerate(train_loader): # loop over examples
    optimizer.zero_grad() # nullify the gradients
    # run data trough the net
    predicted_label = net.forward(data)
    # evaluate the loss
    loss = F.nll_loss(predicted_label, label)
    # backpropagate
    loss.backward()
    # fix weights using the optimizer
    optimizer.step()

    if batch_idx % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
         epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
        train_losses.append(loss.item())
        train_counter.append(
          (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        torch.save(net.state_dict(), 'model.pth')
        torch.save(optimizer.state_dict(), 'optimizer.pth')

def test():
  net.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = net(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [0]:
n_epochs = 3
log_interval = 100

test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()